## Clean buoy data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
buoypath = "../data/plank_ChesapeakeBay_all_buoys_clean.csv"
data = pd.read_csv(buoypath)

In [3]:
def get_valid_value(row, col1, col2, min_valid, max_valid):
    
    if pd.notna(row[col1]) and min_valid <= row[col1] <= max_valid:
        return row[col1]
    elif pd.notna(row[col2]) and min_valid <= row[col2] <= max_valid:
        return row[col2]
    else:
        return np.nan


def merge_valid_if_equal(df, col1, col2, constraint_type, newcol=None, show_diffs=True):
    # Check if the columns are the same
    condition =df[col1] != df[col2]
    df_diffs = df[condition]
    df.loc[condition, col1] = df.loc[condition, col1].fillna(df[col1])
    df.loc[condition, col2] = df.loc[condition, col2].fillna(df[col2])
    # df = df.fillna(-999)
    if show_diffs:
        print(df_diffs[[col1, col2]])
    if newcol is None:
        newcol = constraint_type
    if constraint_type == "Latitude":
        minval, maxval = -90, 90
    if constraint_type == "Longitude":
        minval, maxval = -180, 180
    # if df[col1].equals(df[col2]):
    # Merge the columns into a new column
    df[newcol] = df.apply(get_valid_value, axis=1, args=(col1, col2, minval, maxval))
    df.drop([col1, col2], axis=1, inplace=True)
    return df
    # else:
    #     raise ValueError("The columns are not identical, cannot merge")


# def add_constraints(df, col, constraint_type):
#     if constraint_type == "Latitude":
#         df_filtered = df[(df[col] >= -90) & (df[col] <= 90)]

#     if constraint_type == "Longitude":
#         df_filtered = df[(df[col] >= -180) & (df[col] <= 180)]

#     return df_filtered

In [4]:
data_temp1 = merge_valid_if_equal(data, 'Latitude_x','Latitude_y', 'Latitude')
data_temp2 = merge_valid_if_equal(data_temp1, 'Longitude_x','Longitude_y', 'Longitude')

         Latitude_x  Latitude_y
0          39.20141         NaN
1          39.20141         NaN
2          39.20141         NaN
3           0.00000         NaN
4          39.20142         NaN
...             ...         ...
6222417         NaN         NaN
6222418         NaN         NaN
6222419         NaN         NaN
6222420         NaN         NaN
6222421         NaN         NaN

[3700815 rows x 2 columns]
         Longitude_x  Longitude_y
0          -76.57479          NaN
1          -76.57486          NaN
2          -76.57480          NaN
3                NaN          NaN
4          -76.57483          NaN
...              ...          ...
6222417          NaN          NaN
6222418          NaN          NaN
6222419          NaN          NaN
6222420          NaN          NaN
6222421          NaN          NaN

[3719365 rows x 2 columns]


In [5]:
data_clean = data_temp2.replace([np.inf, -np.inf], np.nan)
missing_percentage = data_clean.isnull().mean()
# print(missing_percentage)
clean_columns = data_clean.columns.tolist()

# Drop columns where more than 90% of the data is missing
columns_to_drop = missing_percentage[missing_percentage > 0.90].index
print(columns_to_drop)
data_clean = data.dropna(subset=['Chlorophyll'])

Index([], dtype='object')


In [6]:
del data
del data_temp1
del data_temp2

In [7]:
#remove outliers
pd.set_option('display.max_columns', None)

# data_clean.min()
# data_clean.max()

# humidity is in percentage and the maximum are in reasonable range, take min(current value, 100)
data_clean['Humidity'] = data_clean['Humidity'].apply(lambda x: min(x, 100.0))

In [8]:
# -50< Air temperature <50
# Air pressure is around 1000 milibars
# Humidity is in percentage
# 0 <= Wind speed < 150
# -50 < Temperature < 50
# 0 < Salinity < 50 (usually 25 is max)
# 0 < Chlorophyll < 500 (usually 100 is max)
# 0 < nephelometric turbidity units
# 0 < oxygen < 20

data_clean = data_clean.drop(data_clean.index[
    (data_clean['Air Temperature']<-50) |
    (data_clean['Air pressure'] < 800) |
    (data_clean['Humidity'] < 0) |
    (data_clean['Wind speed'] < 0) |
    (data_clean['Temperature'] < -50) |
    (data_clean['Salinity'] < 0) |
    (data_clean['Chlorophyll'] < 0) |
    (data_clean['Turbidity'] < 0) |
    (data_clean['Oxygen'] < 0) |
    (data_clean['Air Temperature'] > 50) |
    (data_clean['Air pressure'] > 2000) |
    (data_clean['Wind speed'] > 150) |
    (data_clean['Temperature'] > 50) |
    (data_clean['Salinity'] > 50) |
    (data_clean['Chlorophyll'] > 500) |
    (data_clean['Oxygen'] > 20)
    ].tolist())

# surface currents are not accurate
# data_clean = data_clean.drop(columns= ['North surface currents', 'East surface currents'])

In [9]:
data_clean.columns

Index(['Air Temperature QC', 'Air Temperature', 'Air pressure QC',
       'Air pressure', 'Humidity QC', 'Humidity', 'Wind speed QC',
       'Wind speed', 'Wind Direction QC', 'Wind Direction', 'Temperature QC',
       'Temperature', 'Salinity QC', 'Salinity', 'Chlorophyll QC',
       'Chlorophyll', 'Turbidity QC', 'Turbidity', 'Oxygen QC', 'Oxygen',
       'Waves QC', 'Significant wave height', 'Wave from direction',
       'Wave period', 'North surface currents', 'East surface currents',
       'Sample_year', 'Sample_month', 'Sample_day', 'Sample_hour',
       'Sample_minute', 'Sample_second', 'Latitude', 'Longitude'],
      dtype='object')

In [10]:
# delete QC columns
data_clean = data_clean.drop(columns= ['Air Temperature QC','Air pressure QC','Humidity QC','Wind speed QC','Wind Direction QC', 'Temperature QC','Salinity QC','Chlorophyll QC','Turbidity QC','Oxygen QC','Waves QC'])

In [11]:
data_sorted_buoy = data_clean.sort_values(by=['Sample_year', 'Sample_month', 'Sample_day', 'Sample_hour','Sample_minute'])
# data_sorted_buoy = data_sorted_buoy.reset_index()
del data_clean

data_sorted_buoy.to_csv('../data/to_merge_buoy.csv', index=False)

del data_sorted_buoy

## Clean water quality data

In [15]:
waterqualitypath = "../data/plankton-patrol_ChesapeakeWaterQuality.csv"
data = pd.read_csv(waterqualitypath)

/tmp/ipykernel_58567/2749793111.py:2: DtypeWarning: Columns (1,10,11,12,13,19,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(waterqualitypath)


In [16]:
data["Qualifier"] = data["Qualifier"].replace(np.nan, "=")


columns_to_exclude = ["Parameter", "MeasureValue", "Unit"]
unique_columns = [col for col in data.columns if col not in columns_to_exclude]

df_unique = data[unique_columns].drop_duplicates(subset="EventId")
print(df_unique.shape, data.shape)
data_r = data.pivot_table(
    index=["EventId"], columns="Parameter", values="MeasureValue", aggfunc="first"
).reset_index()
exclude_from_pivoted = ["Parameter", "MeasureValue", "Unit", "SampleDate", "SampleTime"]
pivoted_columns = data_r.columns.tolist()
for ce in exclude_from_pivoted:
    if ce in pivoted_columns:
        pivoted_columns.remove(ce)
data_m = pd.merge(df_unique, data_r, on="EventId", how="left")


print(data_m.columns, data_m.shape)

(20855, 23) (2162314, 26)
Index(['CBSeg2003', 'EventId', 'Cruise', 'Program', 'Project', 'Agency',
       'Source', 'Station', 'SampleDate', 'SampleTime', 'TotalDepth',
       'UpperPycnocline', 'LowerPycnocline', 'Depth', 'Layer', 'SampleType',
       'SampleReplicateType', 'Qualifier', 'Method', 'Lab', 'Latitude',
       'Longitude', 'TierLevel', 'CHLA', 'DIN', 'DO', 'DOC', 'DON', 'DOP',
       'DO_SAT_P', 'FSS', 'KD', 'NH4F', 'NO23F', 'NO2F', 'NO3F', 'PC', 'PH',
       'PHEO', 'PIP', 'PN', 'PO4F', 'PP', 'Parameter', 'SALINITY', 'SECCHI',
       'SIF', 'SIGMA_T', 'SPCOND', 'TDN', 'TDP', 'TN', 'TON', 'TP', 'TSS',
       'TURB_NTU', 'VSS', 'WTEMP'],
      dtype='object') (20855, 58)


In [17]:
data_m.head()

,CBSeg2003,EventId,Cruise,Program,Project,Agency,Source,Station,SampleDate,SampleTime,TotalDepth,UpperPycnocline,LowerPycnocline,Depth,Layer,SampleType,SampleReplicateType,Qualifier,Method,Lab,Latitude,Longitude,TierLevel,CHLA,DIN,DO,DOC,DON,DOP,DO_SAT_P,FSS,KD,NH4F,NO23F,NO2F,NO3F,PC,PH,PHEO,PIP,PN,PO4F,PP,Parameter,SALINITY,SECCHI,SIF,SIGMA_T,SPCOND,TDN,TDP,TN,TON,TP,TSS,TURB_NTU,VSS,WTEMP
0,CB1TF,20021,BAY451,TWQM,MAIN,MDDNR,MDDNR,CB1.1,12/13/2006,10:58:00,7.0,NaN,NaN,6.0,B,D,FS1,=,L01,MDHMH,39.54794,-76.08481,T3,0.598,1.441,12.7,NaN,0.229,0.0084,NaN,NaN,1.2988,0.011,1.43,0.0048,1.4252,0.409,8.0,0.0,NaN,0.0483,0.0097,0.0127,NaN,0.0,0.7,2.8,0.01973,195.0,1.67,0.0181,1.7183,0.2773,0.0308,7.3,NaN,2.0,5.5
1,CB1TF,20236,BAY452,TWQM,MAIN,MDDNR,MDDNR,CB1.1,1/12/2007,13:00:00,5.5,NaN,NaN,4.0,B,D,FS1,=,L01,MDHMH,39.54794,-76.08481,T3,3.489,1.261,13.8,NaN,0.159,0.0079,NaN,NaN,NaN,0.03,1.204,0.0062,1.1975,0.745,7.9,0.0,NaN,0.0858,0.0089,0.0218,NaN,0.0,0.4,2.49,0.02621,137.0,1.41,0.0168,1.4958,0.2498,0.0384,11.3,NaN,5.3,5.3
2,CB1TF,20814,BAY454,TWQM,MAIN,MDDNR,MDDNR,CB1.1,3/14/2007,10:41:00,7.0,NaN,NaN,0.5,S,D,S1,=,L01,MDHMH,39.54794,-76.08481,T3,1.794,1.766,14.4,NaN,0.214,0.0069,NaN,NaN,NaN,0.076,1.62,0.0096,1.5992,0.524,7.7,0.389,NaN,0.075,0.0086,0.0141,NaN,0.0,0.7,2.36,0.02507,234.0,1.98,0.0183,2.055,0.2902,0.0305,7.4,NaN,3.2,2.6
3,CB1TF,6443,BAY456,TWQM,MAIN,MDDNR,MDDNR,CB1.1,4/11/2007,10:32:00,6.5,NaN,NaN,0.5,S,D,S1,=,L01,MDHMH,39.54794,-76.08481,T3,2.99,1.192,12.48,NaN,0.1,0.0023,NaN,NaN,1.2867,0.037,1.157,0.0099,1.1475,0.447,7.9,0.0,NaN,0.0653,0.0075,0.015,NaN,0.0,0.8,2.03,-0.05797,185.0,1.35,0.0098,1.4153,0.1653,0.0237,8.0,NaN,2.8,7.6
4,CB1TF,6767,BAY457,TWQM,MAIN,MDDNR,MDDNR,CB1.1,4/26/2007,11:46:00,5.5,NaN,NaN,0.5,S,D,S1,=,L01,MDHMH,39.54794,-76.08481,T3,2.392,1.023,9.76,NaN,0.017,0.007,NaN,NaN,1.9807,0.026,0.983,0.0079,0.9749,0.636,7.5,0.635,NaN,0.0778,0.0068,0.0197,NaN,0.0,0.5,1.81,-0.67801,142.0,1.04,0.0132,1.1297,0.1568,0.0331,15.0,NaN,3.0,13.9


In [18]:
columns_to_drop = ['EventId','CBSeg2003','Cruise','Program','Project','Agency','Source','Lab','TierLevel']

# Drop the specified columns
data_md = data_m.drop(columns=columns_to_drop)


In [19]:
set_numeric = [
    "TotalDepth",
    "UpperPycnocline",
    "LowerPycnocline",
    "Depth",
    "Latitude",
    "Longitude",
    "MeasureValue",
    "CHLA",
    "DIN",
    "DO",
    "DOC",
    "DON",
    "DOP",
    "DO_SAT_P",
    "FSS",
    "KD",
    "NH4F",
    "NO23F",
    "NO2F",
    "NO3F",
    "PC",
    "PH",
    "PHEO",
    "PIP",
    "PN",
    "PO4F",
    "PP",
    "Parameter",
    "SALINITY",
    "SECCHI",
    "SIF",
    "SIGMA_T",
    "SPCOND",
    "TDN",
    "TDP",
    "TN",
    "TON",
    "TP",
    "TSS",
    "TURB_NTU",
    "VSS",
    "WTEMP",
]
# set_string = ['
set_date = ["SampleDate", "SampleTime"]


def combine_date_time_strings(df, date_col, time_col):
    # Combine date and time strings
    combined_col = df[date_col] + " " + df[time_col]
    # Convert the combined string to datetime
    datetime_col = "Sample"

    df[datetime_col] = pd.to_datetime(combined_col, errors="coerce")
    df[datetime_col + "_year"] = df[datetime_col].dt.year
    df[datetime_col + "_month"] = df[datetime_col].dt.month
    df[datetime_col + "_day"] = df[datetime_col].dt.day
    df[datetime_col + "_hour"] = df[datetime_col].dt.hour
    df[datetime_col + "_minute"] = df[datetime_col].dt.minute
    df[datetime_col + "_second"] = df[datetime_col].dt.second
    newcolnames = [
        f"{datetime_col}_year",
        f"{datetime_col}_month",
        f"{datetime_col}_day",
        f"{datetime_col}_hour",
        f"{datetime_col}_minute",
        f"{datetime_col}_second",
    ]
    # Drop the original date, time, and combined datetime columns
    df.drop(columns=[date_col, time_col, datetime_col], inplace=True)

    return df, newcolnames


# Function to convert columns to appropriate types
def convert_dtypes(df):
    for col in df.columns:
        df[col] = df[col].replace("nan", np.nan)
        # print(df[col][564463])
        print(f"converting column {col}", end="\t")
        print(df[col].dtype)

        if col not in (set_numeric + set_date):
            df[col] = pd.Categorical(df[col])
            print("Categorical")
        elif col in set_numeric:
            # try:
            # Try converting to numeric (float)
            df[col] = pd.to_numeric(df[col], errors="coerce")
            print("Numeric")
        # except (ValueError, TypeError):
        # try:
        elif col in set_date:
            print("Date, skipped")
            pass
            # Try converting to datetime (date)
            # df[col] = pd.to_datetime(df[col], errors='coerce')
            # except (ValueError, TypeError):
            # Check for categorical
            # unique_ratio = df[col].nunique() / df[col].count()
            # if unique_ratio < 0.2:  # heuristic for categorical, adjustable threshold
            # Convert to string if not numeric, date, or categorical
        else:
            print(f"{col}: string")
            df[col] = df[col].astype(str)
            print("string")
        print(df[col].dtype)
    return df


# Apply the conversion function
datacopy = data_md.__deepcopy__()
data_conv = convert_dtypes(datacopy)
data_cleana, newcols = combine_date_time_strings(data_conv, "SampleDate", "SampleTime")
# Check the result
print(data_cleana.dtypes)
print(data_cleana.shape)
# data.dropna(axis='TotalDepth',how='all')
data_cleana = data_cleana[:-1]
print(data_cleana.shape)

# data_conv = convert_dtypes(data_conv)
# for col in data.columns:
#     # print(data[col])
#     if col not in (set_float + set_date):
#         data[col]= pd.Categorical(df[col])(data[col][1:])
#     # if col in set_string:
#     #     data[col][1:]=data[col][1:].astype("string")
#     if col in set_float:
#         data[col]= pd.to_numeric(data[col], errors='raise')
#     if col in set_date:
#         print(col)
#         data[col][1:]=pd.to_datetime(data[col][1:],errors='coerce')
#         # data[col]=data[col].dt.strftime(f'%m/%d/%Y')
data_cleana

converting column Station	object
Categorical
category
converting column SampleDate	object
Date, skipped
object
converting column SampleTime	object
Date, skipped
object
converting column TotalDepth	object
Numeric
float64
converting column UpperPycnocline	object
Numeric
float64
converting column LowerPycnocline	object
Numeric
float64
converting column Depth	object
Numeric
float64
converting column Layer	object
Categorical
category
converting column SampleType	object
Categorical
category
converting column SampleReplicateType	object
Categorical
category
converting column Qualifier	object
Categorical
category
converting column Method	object
Categorical
category
converting column Latitude	object
Numeric
float64
converting column Longitude	object
Numeric
float64
converting column CHLA	object
Numeric
float64
converting column DIN	object
Numeric
float64
converting column DO	object
Numeric
float64
converting column DOC	object
Numeric
float64
converting column DON	object
Numeric
float64
convertin

,Station,TotalDepth,UpperPycnocline,LowerPycnocline,Depth,Layer,SampleType,SampleReplicateType,Qualifier,Method,Latitude,Longitude,CHLA,DIN,DO,DOC,DON,DOP,DO_SAT_P,FSS,KD,NH4F,NO23F,NO2F,NO3F,PC,PH,PHEO,PIP,PN,PO4F,PP,Parameter,SALINITY,SECCHI,SIF,SIGMA_T,SPCOND,TDN,TDP,TN,TON,TP,TSS,TURB_NTU,VSS,WTEMP,Sample_year,Sample_month,Sample_day,Sample_hour,Sample_minute,Sample_second
0,CB1.1,7.0,NaN,NaN,6.0,B,D,FS1,=,L01,39.54794,-76.08481,0.598,1.441,12.70,NaN,0.229,0.0084,NaN,NaN,1.2988,0.011,1.430,0.0048,1.4252,0.409,8.0,0.000,NaN,0.0483,0.0097,0.0127,NaN,0.00,0.7,2.80,0.01973,195.0,1.67,0.0181,1.7183,0.2773,0.0308,7.3,NaN,2.0,5.5,2006.0,12.0,13.0,10.0,58.0,0.0
1,CB1.1,5.5,NaN,NaN,4.0,B,D,FS1,=,L01,39.54794,-76.08481,3.489,1.261,13.80,NaN,0.159,0.0079,NaN,NaN,NaN,0.030,1.204,0.0062,1.1975,0.745,7.9,0.000,NaN,0.0858,0.0089,0.0218,NaN,0.00,0.4,2.49,0.02621,137.0,1.41,0.0168,1.4958,0.2498,0.0384,11.3,NaN,5.3,5.3,2007.0,1.0,12.0,13.0,0.0,0.0
2,CB1.1,7.0,NaN,NaN,0.5,S,D,S1,=,L01,39.54794,-76.08481,1.794,1.766,14.40,NaN,0.214,0.0069,NaN,NaN,NaN,0.076,1.620,0.0096,1.5992,0.524,7.7,0.389,NaN,0.0750,0.0086,0.0141,NaN,0.00,0.7,2.36,0.02507,234.0,1.98,0.0183,2.0550,0.2902,0.0305,7.4,NaN,3.2,2.6,2007.0,3.0,14.0,10.0,41.0,0.0
3,CB1.1,6.5,NaN,NaN,0.5,S,D,S1,=,L01,39.54794,-76.08481,2.990,1.192,12.48,NaN,0.100,0.0023,NaN,NaN,1.2867,0.037,1.157,0.0099,1.1475,0.447,7.9,0.000,NaN,0.0653,0.0075,0.0150,NaN,0.00,0.8,2.03,-0.05797,185.0,1.35,0.0098,1.4153,0.1653,0.0237,8.0,NaN,2.8,7.6,2007.0,4.0,11.0,10.0,32.0,0.0
4,CB1.1,5.5,NaN,NaN,0.5,S,D,S1,=,L01,39.54794,-76.08481,2.392,1.023,9.76,NaN,0.017,0.0070,NaN,NaN,1.9807,0.026,0.983,0.0079,0.9749,0.636,7.5,0.635,NaN,0.0778,0.0068,0.0197,NaN,0.00,0.5,1.81,-0.67801,142.0,1.04,0.0132,1.1297,0.1568,0.0331,15.0,NaN,3.0,13.9,2007.0,4.0,26.0,11.0,46.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20849,MDE.1802025,NaN,NaN,NaN,11.0,VP,ISM,M1,=,F01,38.03722,-75.92444,NaN,NaN,3.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.9,2019.0,8.0,21.0,12.0,19.0,0.0
20850,MDE.1802025,NaN,NaN,NaN,0.5,VP,ISM,M1,=,F01,38.03722,-75.92444,NaN,NaN,6.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.9,2019.0,9.0,16.0,12.0,10.0,0.0
20851,MDE.1802025,NaN,NaN,NaN,9.0,VP,ISM,M1,=,F01,38.03722,-75.92444,NaN,NaN,6.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.3,2022.0,7.0,6.0,10.0,10.0,0.0
20852,MDE.1802025,NaN,NaN,NaN,8.0,VP,ISM,M1,=,F01,38.03722,-75.92444,NaN,NaN,5.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.7,2022.0,8.0,3.0,12.0,42.0,0.0


In [20]:
pivoted_columns= pivoted_columns+newcols
print(len(pivoted_columns), pivoted_columns)

41 ['EventId', 'CHLA', 'DIN', 'DO', 'DOC', 'DON', 'DOP', 'DO_SAT_P', 'FSS', 'KD', 'NH4F', 'NO23F', 'NO2F', 'NO3F', 'PC', 'PH', 'PHEO', 'PIP', 'PN', 'PO4F', 'PP', 'SALINITY', 'SECCHI', 'SIF', 'SIGMA_T', 'SPCOND', 'TDN', 'TDP', 'TN', 'TON', 'TP', 'TSS', 'TURB_NTU', 'VSS', 'WTEMP', 'Sample_year', 'Sample_month', 'Sample_day', 'Sample_hour', 'Sample_minute', 'Sample_second']


In [21]:
# pivoted_columns = [col for col in pivoted_columns if col in data_clean.columns]

# data_clean = data_clean.replace([np.inf, -np.inf], np.nan).dropna(
#     subset=[pivoted_columns]
# )
data_cleana = data_cleana.replace([np.inf, -np.inf], np.nan)


missing_percentage = data_cleana.isnull().mean()
print(missing_percentage)
clean_columns = data_cleana.columns.tolist()

# Drop columns where more than 95% of the data is missing
columns_to_drop = missing_percentage[missing_percentage > 0.90].index
print(columns_to_drop)
data_cleana = data_cleana.drop(columns=columns_to_drop)
data_clean = data_cleana.copy()
data_clean = data_cleana.dropna(subset=['CHLA'])
# data_clean = data_cleana.dropna(
#     subset=[
#         "CHLA",
#         "DIN",
#         "DO",
#         "DOC",
#         "DON",
#         "DOP",
#         "DO_SAT_P",
#         "FSS",
#         "KD",
#         "NH4F",
#         "NO23F",
#         "NO2F",
#         "NO3F",
#         "PC",
#         "PH",
#         "PHEO",
#         "PN",
#         "PO4F",
#         "PP",
#         "SALINITY",
#         "SECCHI",
#         "SIF",
#         "SIGMA_T",
#         "SPCOND",
#         "TDN",
#         "TDP",
#         "TN",
#         "TON",
#         "TP",
#         "TSS",
#         "VSS",
#         "WTEMP",
#     ]
# )
dropped_columns = [col for col in clean_columns if col not in data_clean.columns]

data_clean.info()

Station                0.000048
TotalDepth             0.036636
UpperPycnocline        0.761485
LowerPycnocline        0.761485
Depth                  0.000096
Layer                  0.000144
SampleType             0.000144
SampleReplicateType    0.000048
Qualifier              0.000000
Method                 0.000048
Latitude               0.000096
Longitude              0.000096
CHLA                   0.177951
DIN                    0.390908
DO                     0.070682
DOC                    0.890860
DON                    0.394505
DOP                    0.392730
DO_SAT_P               0.821617
FSS                    0.565743
KD                     0.321905
NH4F                   0.388702
NO23F                  0.388702
NO2F                   0.388702
NO3F                   0.390956
PC                     0.388990
PH                     0.068668
PHEO                   0.183418
PIP                    0.954925
PN                     0.389661
PO4F                   0.388511
PP      

In [22]:
print(dropped_columns)

['PIP', 'Parameter', 'TURB_NTU']


In [28]:
data_clean.columns

Index(['Station', 'TotalDepth', 'UpperPycnocline', 'LowerPycnocline', 'Depth',
       'Layer', 'SampleType', 'SampleReplicateType', 'Qualifier', 'Method',
       'Latitude', 'Longitude', 'CHLA', 'DIN', 'DO', 'DOC', 'DON', 'DOP',
       'DO_SAT_P', 'FSS', 'KD', 'NH4F', 'NO23F', 'NO2F', 'NO3F', 'PC', 'PH',
       'PHEO', 'PN', 'PO4F', 'PP', 'SALINITY', 'SECCHI', 'SIF', 'SIGMA_T',
       'SPCOND', 'TDN', 'TDP', 'TN', 'TON', 'TP', 'TSS', 'VSS', 'WTEMP',
       'Sample_year', 'Sample_month', 'Sample_day', 'Sample_hour',
       'Sample_minute', 'Sample_second'],
      dtype='object')

In [33]:
columns_to_convert = ['Sample_year', 'Sample_month', 'Sample_day', 'Sample_hour', 'Sample_minute', 'Sample_second']
data_clean[columns_to_convert] = data_clean[columns_to_convert].astype(int)

/tmp/ipykernel_58567/1494001152.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean[columns_to_convert] = data_clean[columns_to_convert].astype(int)


In [40]:
data_clean.drop(columns=['Station','Layer', 'SampleType', 'SampleReplicateType','Qualifier','Method'])

,TotalDepth,UpperPycnocline,LowerPycnocline,Depth,Latitude,Longitude,CHLA,DIN,DO,DOC,DON,DOP,DO_SAT_P,FSS,KD,NH4F,NO23F,NO2F,NO3F,PC,PH,PHEO,PN,PO4F,PP,SALINITY,SECCHI,SIF,SIGMA_T,SPCOND,TDN,TDP,TN,TON,TP,TSS,VSS,WTEMP,Sample_year,Sample_month,Sample_day,Sample_hour,Sample_minute,Sample_second
0,7.0,NaN,NaN,6.0,39.54794,-76.08481,0.598,1.441,12.70,NaN,0.229,0.0084,NaN,NaN,1.29880,0.011,1.430,0.0048,1.4252,0.409,8.0,0.000,0.0483,0.0097,0.0127,0.00,0.7,2.80,0.01973,195.0,1.67,0.0181,1.7183,0.2773,0.0308,7.3,2.0,5.5,2006,12,13,10,58,0
1,5.5,NaN,NaN,4.0,39.54794,-76.08481,3.489,1.261,13.80,NaN,0.159,0.0079,NaN,NaN,NaN,0.030,1.204,0.0062,1.1975,0.745,7.9,0.000,0.0858,0.0089,0.0218,0.00,0.4,2.49,0.02621,137.0,1.41,0.0168,1.4958,0.2498,0.0384,11.3,5.3,5.3,2007,1,12,13,0,0
2,7.0,NaN,NaN,0.5,39.54794,-76.08481,1.794,1.766,14.40,NaN,0.214,0.0069,NaN,NaN,NaN,0.076,1.620,0.0096,1.5992,0.524,7.7,0.389,0.0750,0.0086,0.0141,0.00,0.7,2.36,0.02507,234.0,1.98,0.0183,2.0550,0.2902,0.0305,7.4,3.2,2.6,2007,3,14,10,41,0
3,6.5,NaN,NaN,0.5,39.54794,-76.08481,2.990,1.192,12.48,NaN,0.100,0.0023,NaN,NaN,1.28670,0.037,1.157,0.0099,1.1475,0.447,7.9,0.000,0.0653,0.0075,0.0150,0.00,0.8,2.03,-0.05797,185.0,1.35,0.0098,1.4153,0.1653,0.0237,8.0,2.8,7.6,2007,4,11,10,32,0
4,5.5,NaN,NaN,0.5,39.54794,-76.08481,2.392,1.023,9.76,NaN,0.017,0.0070,NaN,NaN,1.98070,0.026,0.983,0.0079,0.9749,0.636,7.5,0.635,0.0778,0.0068,0.0197,0.00,0.5,1.81,-0.67801,142.0,1.04,0.0132,1.1297,0.1568,0.0331,15.0,3.0,13.9,2007,4,26,11,46,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20813,1.3,NaN,NaN,0.5,38.13318,-76.07493,4.913,NaN,7.50,NaN,NaN,NaN,NaN,NaN,1.07281,NaN,NaN,NaN,NaN,NaN,8.1,0.740,NaN,NaN,NaN,13.99,0.7,NaN,8.11479,23400.0,NaN,NaN,NaN,NaN,NaN,12.4,NaN,23.3,2023,9,20,10,6,0
20814,3.3,NaN,NaN,0.5,38.06203,-75.91400,10.466,NaN,9.40,NaN,NaN,NaN,NaN,NaN,1.06244,NaN,NaN,NaN,NaN,NaN,8.2,0.748,NaN,NaN,NaN,18.48,1.5,NaN,12.52473,30000.0,NaN,NaN,NaN,NaN,NaN,54.0,NaN,19.2,2023,10,11,10,52,0
20815,3.2,NaN,NaN,0.5,38.06203,-75.91400,8.330,NaN,7.30,NaN,NaN,NaN,NaN,NaN,1.08680,NaN,NaN,NaN,NaN,NaN,8.1,1.239,NaN,NaN,NaN,18.97,1.0,NaN,11.51797,30600.0,NaN,NaN,NaN,NaN,NaN,6.4,NaN,24.4,2023,9,20,10,43,0
20816,10.2,NaN,NaN,0.5,37.94350,-75.92480,12.175,NaN,8.40,NaN,NaN,NaN,NaN,NaN,0.79377,NaN,NaN,NaN,NaN,NaN,8.0,1.282,NaN,NaN,NaN,18.97,1.3,NaN,12.94351,30700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,2023,10,11,9,31,0


In [41]:
data_sorted_water = data_clean.sort_values(by=['Sample_year', 'Sample_month', 'Sample_day', 'Sample_hour','Sample_minute','Sample_second'])

In [42]:
data_sorted_water

,Station,TotalDepth,UpperPycnocline,LowerPycnocline,Depth,Layer,SampleType,SampleReplicateType,Qualifier,Method,Latitude,Longitude,CHLA,DIN,DO,DOC,DON,DOP,DO_SAT_P,FSS,KD,NH4F,NO23F,NO2F,NO3F,PC,PH,PHEO,PN,PO4F,PP,SALINITY,SECCHI,SIF,SIGMA_T,SPCOND,TDN,TDP,TN,TON,TP,TSS,VSS,WTEMP,Sample_year,Sample_month,Sample_day,Sample_hour,Sample_minute,Sample_second
2160,HOK0005,1.5,NaN,NaN,0.5,BS,D,S1,=,L01,38.18890,-76.26530,35.8850,0.0338,6.60,4.530,0.3662,0.0118,NaN,NaN,3.92310,0.0140,0.0198,0.0049,0.0149,3.95000,7.10,7.72500,0.61400,0.0053,0.0648,0.55,0.3,2.6400,-3.03805,1770.0,0.4000,0.0171,1.01400,0.98020,0.0819,40.70,10.00,27.100,2004,7,29,10,38,0
3700,XGF0681,0.8,NaN,NaN,0.5,BS,D,S1,=,L01,38.84400,-76.36530,12.7090,0.0133,9.20,3.440,0.3467,0.0089,NaN,NaN,1.40790,0.0070,0.0063,0.0012,0.0051,2.10000,8.38,1.68200,0.39700,0.0050,0.0263,10.12,0.6,0.9100,4.18306,17519.0,0.3600,0.0139,0.75700,0.74370,0.0402,13.50,4.00,27.000,2004,7,29,11,15,0
1853,CB5.1W,9.0,NaN,NaN,0.5,S,D,FS1,=,L01,38.32522,-76.37574,21.2300,0.0531,7.90,2.990,0.5336,0.0107,NaN,NaN,NaN,0.0060,0.0034,0.0014,0.0030,1.90000,8.30,3.17700,0.38600,0.0088,0.0334,12.93,0.8,0.7600,6.34847,21800.0,0.3700,0.0177,0.74500,0.91960,0.0471,11.70,NaN,26.800,2004,8,2,9,45,0
3276,CB8.1,9.0,NaN,NaN,1.0,S,D,S1,=,L01,36.99542,-76.16772,13.8840,0.0968,7.73,NaN,0.2042,0.0082,NaN,6.40,0.68640,0.0175,0.0116,0.0044,0.0038,1.16006,7.98,2.37630,0.11380,0.0077,0.0161,18.52,1.1,0.5346,14.50101,30253.0,0.3010,0.0159,0.41480,0.45430,0.0395,8.10,NaN,24.950,2004,8,9,7,30,0
3277,CB8.1E,12.0,NaN,NaN,1.0,S,D,S1,=,L01,36.94737,-76.03494,11.8815,0.0591,8.13,NaN,0.2099,0.0074,NaN,4.54,0.72430,0.0109,0.0117,0.0006,0.0042,1.10397,8.08,1.66875,0.13314,0.0046,0.0150,19.90,1.2,0.3713,13.34482,32053.0,0.2690,0.0120,0.40210,0.41310,0.0322,8.20,NaN,24.940,2004,8,9,8,41,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18126,CB5.5,16.0,4.0,9.0,1.0,S,D,S1,=,L01,37.69180,-76.18967,13.3500,0.1929,10.73,NaN,0.2242,0.0016,NaN,1.60,0.44250,0.0038,0.1891,0.0057,0.1834,1.21453,7.95,0.67600,0.16235,0.0014,0.0092,NaN,1.9,0.0122,NaN,NaN,0.4171,0.0030,0.57796,0.38506,0.0122,4.94,3.34,14.333,2024,4,17,11,53,0
18246,CB5.4W,5.0,NaN,NaN,1.0,S,D,S1,=,L01,37.81347,-76.29467,14.4180,0.1357,9.91,NaN,0.2327,0.0027,NaN,2.46,0.51888,0.0038,0.1319,0.0054,0.1265,1.36868,8.29,0.67600,0.18023,0.0016,0.0113,NaN,1.7,0.0143,NaN,NaN,0.3684,0.0043,0.54863,0.41293,0.0153,6.02,3.56,16.131,2024,4,17,12,39,0
18124,CB5.4,33.0,7.0,17.0,1.0,S,D,S1,=,L01,37.80013,-76.17466,18.0492,0.2086,11.46,NaN,0.2473,0.0021,NaN,2.04,0.55690,0.0038,0.2048,0.0058,0.1990,1.47902,8.19,0.67600,0.15884,0.0014,0.0223,NaN,1.6,0.0156,NaN,NaN,0.3287,0.0051,0.61441,0.40581,0.0137,5.54,3.50,10.681,2024,4,17,13,6,0
18723,CB6.1,13.0,8.0,9.0,1.0,S,D,S1,=,L01,37.58847,-76.16216,12.9228,0.1538,7.66,2.757,0.2248,0.0020,NaN,1.44,0.60459,0.0038,0.1500,0.0046,0.1454,0.81153,8.11,0.67600,0.11752,0.0014,0.0097,NaN,1.6,0.0107,NaN,NaN,0.3786,0.0034,0.54703,0.39323,0.0131,4.88,3.44,12.276,2024,4,17,14,12,0


In [43]:
data_sorted_water.to_csv('../data/to_merge_water.csv', index=False)

## Toxic data (deprecated)

In [67]:
combinedbuoyspath = "../data/ToxicsChemicalContaminantHUC8_updated.csv"
# New file with deleted row no 131050, for 'new' file row no 275590

# data = pd.read_csv(combinedbuoyspath, on_bad_lines='skip')
data = pd.read_csv(combinedbuoyspath,sep=',')
# data.info()

/tmp/ipykernel_58567/3502890825.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(combinedbuoyspath,sep=',')


In [71]:
data['SampleDateTime']

0         1997-08-11 23:59:00
1         1997-08-11 23:59:00
2         1997-08-11 23:59:00
3         1997-08-11 23:59:00
4         1997-08-11 23:59:00
                 ...         
176949    2000-09-30 23:59:00
176950    2000-09-30 23:59:00
176951    2000-09-30 23:59:00
176952                    NaN
176953                    NaN
Name: SampleDateTime, Length: 176954, dtype: object

In [55]:
data_r = data.pivot_table(
    index=["SampleDateTime"], columns="ChemicalName", values="ReportedValue", aggfunc="first"
).reset_index()

In [56]:
pivotcols = data_r.columns
print(pivotcols.shape)
data_r

(744,)


ChemicalName,SampleDateTime,% LIPID,% MOISTURE,"1,1,1-TRICHLOROETHANE","1,1,2,2-TETRACHLOROETHANE","1,1,2-TRICHLOROETHANE","1,1-DICHLOROETHANE","1,2,3,4-TETRACHLOROBENZENE","1,2,4,5-TETRACHLOROBENZENE","1,2,4-TRICHLOROBENZENE","1,2,7-TRIMETHYLNAPHTHALENE","1,2-DICHLOROBENZENE","1,2-DICHLOROETHANE","1,2-DICHLOROPROPANE","1,3-DICHLOROBENZENE","1,4-DIATHIANE","1,4-DICHLOROBENZENE","1,4-OXYTHIANE",1-METHYLNAPHTHALENE,1-METHYLPHENANTHRENE,1-METHYLPYRENE,"17a, 21b (H)-Hopane","2,2,4,4 TRICHLOROBIPHENYL","2,2,4,4-TETRACHLOROBIPHENYL","2,3,4,4'-TETRACHLOROBIPHENYL","2,3,5-TRIMETHYLNAPHTHALENE","2,4,5-T","2,4,5-TP - PHENOXY ACID","2,4,5-TRICHLOROPHENOL","2,4,6-TRICHLOROPHENOL","2,4-D","2,4-DB","2,4-DICHLOROPHENOL","2,4-DIMETHYLPHENOL","2,4-DINITROPHENOL","2,4-DINITROTOLUENE","2,4-DP (DICHLORPROP)","2,6-DIETHYLANILE","2,6-DIMETHYLNAPHTHALENE","2,6-DINITROTOLUENE",2-CHLORONAPHTHALENE,2-CHLOROPHENOL,2-HEXANONE,"2-METHYL-4,6-DINITROPHENOL",2-METHYLFLUORANTHENE,2-METHYLNAPHTHALENE,2-METHYLPHENANTHRENE,2-NITROANILINE,2-NITROPHENOL,"3,3'-DICHLOROBENZIDINE",3-HYDROXYCARBOFURAN,3-METHYLPHENANTHRENE,3-NITROANILINE,4-BROMOPHENYL PHENYL ETHER,4-CHLORO-3-METHYLPHENOL,4-CHLOROANILINE,4-CHLOROPHENYL PHENYL ETHER,4-METHYL PHENOL,4-METHYLCHRYSENE,4-NITROANILINE,4-NITROPHENOL,4H-CYCLOPENTA[DEF]PHENANTHRENE,ACENAPHTHENE,ACENAPHTHYLENE,ACETOCHLOR,ACETONE,ACID VOLATILE SULFIDE,ALACHLOR,ALDICARB,ALDICARB SULFONE,ALDICARB SULFOXIDE,ALDRIN,ALUMINUM,AMETRYN,ANILINE,ANTHRACENE,ANTIMONY,AROCHLOR,"AROCLOR, PCB",ARSENIC,ARSENIC (III+V),ARSENITE (As III),ATRAZINE,AVS MINUS SEM,AZINPHOS METHYL,AZOBENZENE,Aroclor 5460,BARIUM,BDE (TOTAL),BDE-100,BDE-100 + PCB-194,BDE-153,BDE-154,BDE-47,BDE-99,BENEFIN,BENZENE,BENZIDINE,BENZO(B+K)FLUORANTHENE,BENZOIC ACID,BENZONAPHTHOTHIOPHENE,BENZO[A]FLUORENE,BENZO[A]PYRENE,BENZO[B]FLUORANTHENE,BENZO[B]FLUORENE,BENZO[C]PHENANTHRENE,BENZO[E]PYRENE,BENZO[GHI]FLUORANTHENE,BENZO[GHI]PERYLENE,BENZO[J]FLUORANTHENE,BENZO[K]FLUORANTHENE,BENZYL ALCOHOL,BENZ[A]ANTHRACENE,BERYLLIUM,BHC-ALPHA,BHC-BETA,BHC-DELTA,BHC-GAMMA (LINDANE),BIBENZYlL,BILE BENZO,BILE NAPH,BILE PHEN,BILE TPAH,BIPHENYL,"BIPHENYL, 4-METHYL",BIS (2-CHLOROISOPROPYL) ETHER,"BIS(2,4,6-TRICHLOROPHENYL) UREA (TCPU)",BIS(2-CHLOROETHOXY)METHANE,BIS(2-CHLOROETHYL)ETHER,BIS(TRI-N-BUTYLTIN)OXIDE,BORON,"BROMKAL,PENTABROMO-",BROMODICHLOROMETHANE,BROMOFORM,BUTYL BENZYL PHTHALATE,BUTYLATE,"Benzenehexachloride,alpha-","Benzenehexachloride,beta-","Benzenehexachloride,gamma-",C1 CHRYSENE,C1 DIBENZOTH,C1 FLUORENE,"C1-Dibenzo(a,h)anthracene",C1-Fluoranthenes & Pyrenes,C1-NAPHTHALENE,C1-Naphthobenzothiophene,C1-Phenanthrenes & Anthracenes,C1_Fluoranthene/Anthracene,C2 CHRYSENE,C2 DIBENZOTH,C2 FLUORENE,"C2-Dibenzo(a,h)anthracene",C2-Fluoranthenes/Pyrenes,C2-NAPHTHALENE,C2-Naphthobenzothiophene,C2-Phenanthrenes & Anthracenes,C2_Fluoranthene/Anthracene,C3 CHRYSENE,C3 DIBENZOTH,"C3-Dibenzo(a,h)anthracene",C3-FLUORENE,C3-Fluoranthenes/Pyrenes,C3-NAPHTHALENE,C3-Naphthobenzothiophene,C3-Phenanthrenes & Anthracenes,C4 CHRYSENE,C4-NAPHTHALENE,C4-Phenanthrenes & Anthracenes,CADMIUM,CALCIUM,CALCIUM CARBONATE,CARBARYL (SEVIN),CARBAZOLE,CARBOFURAN,CARBON,CARBON DISULFIDE,CARBON TETRACHLORIDE,CARBOPHENOTHION,CHLORDANE,CHLORDANE MIX,CHLORDANE-ALPHA,CHLORDANE-CIS,CHLORDANE-GAMMA,CHLORDANE-TRANS,CHLORIDE,CHLOROBENZENE,"CHLOROBIPHENYL, 2,3,3,4,4',5,5","CHLOROBIPHENYL, POLY-",CHLOROETHANE,CHLOROFORM,CHLOROMETHANE,CHLOROTHALONIL,CHLORPYRIFOS,CHLORPYRIFOS-METHYL,CHROMIUM,CHROMIUM (VI),"CHROMIUM III, TOTAL",CHRYSENE,CHRYSENE PLUS TRIPHENYLENE,CHRYSENE-BENZ[A]ANTHRACENE,CIS- AND TRANS-PERMETHRIN,"CIS-1,3-DICHLOROPROPENE",CIS-PERMETHRIN,"CLAY, P.S. = <.0039",COBALT,COPPER,CYANAZINE,Chlordane (1),Chlordane C,Chlordane cpd,"Chlordane cpd + DDE,2,4'- + PCB-84","Chlordane cpd + PCB-37,42,59",Chlordane cpd + PCB-44,Chlordane cpd + PCB-84,Chlordane(1),Chlordane(3),Chlordane(5),"Chlordane(5) + DDE,2,4'- + PCB-84",Chlordane(5) + PCB-84,Chlordane(7),"Chlordane, cis- + PCB-99","Chlordane, trans- + PCB-84","Chlordane, trans- + PCB-9

In [57]:
def combine_date_time_strings(df, originaldtcolumn):
    # Combine date and time strings
    combined_col = df[originaldtcolumn]
    # Convert the combined string to datetime
    datetime_col = "Sample"

    df[datetime_col] = pd.to_datetime(combined_col, errors="coerce")
    df[datetime_col + "_year"] = df[datetime_col].dt.year
    df[datetime_col + "_month"] = df[datetime_col].dt.month
    df[datetime_col + "_day"] = df[datetime_col].dt.day
    df[datetime_col + "_hour"] = df[datetime_col].dt.hour
    df[datetime_col + "_minute"] = df[datetime_col].dt.minute
    df[datetime_col + "_second"] = df[datetime_col].dt.second
    newcolnames = [
        f"{datetime_col}_year",
        f"{datetime_col}_month",
        f"{datetime_col}_day",
        f"{datetime_col}_hour",
        f"{datetime_col}_minute",
        f"{datetime_col}_second",
    ]
    # Drop the original date, time, and combined datetime columns
    df.drop(columns=[originaldtcolumn, datetime_col], inplace=True)

    return df, newcolnames

In [58]:
columns_to_exclude = ["ChemicalName", "ReportedValue", "Unit", "Qualifier"]
unique_columns = [col for col in data.columns if col not in columns_to_exclude]
df_unique = data[unique_columns].drop_duplicates(subset="SampleDateTime")
exclude_from_pivoted = ["ChemicalName", "ReportedValue", "Unit", "Qualifier"]
data_cleana, newcols = combine_date_time_strings(data_r, "SampleDateTime")
newcols


['Sample_year',
 'Sample_month',
 'Sample_day',
 'Sample_hour',
 'Sample_minute',
 'Sample_second']

In [61]:
data_cleana = data_cleana.replace([np.inf, -np.inf], np.nan)
missing_percentage = data_cleana.isnull().mean()
# print(missing_percentage)
clean_columns = data_cleana.columns.tolist()

# Drop columns where more than 95% of the data is missing
columns_to_drop = missing_percentage[missing_percentage > 0.90].index
print(columns_to_drop)
# data_clean = data_clean.dropna(subset=['Chlorophyll'])

Index(['% LIPID', '% MOISTURE', '1,1,1-TRICHLOROETHANE',
       '1,1,2,2-TETRACHLOROETHANE', '1,1,2-TRICHLOROETHANE',
       '1,1-DICHLOROETHANE', '1,2,3,4-TETRACHLOROBENZENE',
       '1,2,4,5-TETRACHLOROBENZENE', '1,2,4-TRICHLOROBENZENE',
       '1,2,7-TRIMETHYLNAPHTHALENE',
       ...
       'TRIPENTYLTIN', 'URANIUM', 'Unknown tetra-BDE',
       'Unknown tetra-BDE + PCB-171,156', 'VANADIUM', 'VINYL CHLORIDE',
       'VINYLIDENE CHLORIDE', 'VINYLIDENE CHLORIDE, TOTAL', 'WEIGHT',
       'XYLENE (MIXED ISOMERS)'],
      dtype='object', name='ChemicalName', length=727)


In [64]:
data_sorted_toxic = data_cleana.sort_values(by=['Sample_year', 'Sample_month', 'Sample_day', 'Sample_hour','Sample_minute','Sample_second'])

In [65]:
data_sorted_toxic

ChemicalName,% LIPID,% MOISTURE,"1,1,1-TRICHLOROETHANE","1,1,2,2-TETRACHLOROETHANE","1,1,2-TRICHLOROETHANE","1,1-DICHLOROETHANE","1,2,3,4-TETRACHLOROBENZENE","1,2,4,5-TETRACHLOROBENZENE","1,2,4-TRICHLOROBENZENE","1,2,7-TRIMETHYLNAPHTHALENE","1,2-DICHLOROBENZENE","1,2-DICHLOROETHANE","1,2-DICHLOROPROPANE","1,3-DICHLOROBENZENE","1,4-DIATHIANE","1,4-DICHLOROBENZENE","1,4-OXYTHIANE",1-METHYLNAPHTHALENE,1-METHYLPHENANTHRENE,1-METHYLPYRENE,"17a, 21b (H)-Hopane","2,2,4,4 TRICHLOROBIPHENYL","2,2,4,4-TETRACHLOROBIPHENYL","2,3,4,4'-TETRACHLOROBIPHENYL","2,3,5-TRIMETHYLNAPHTHALENE","2,4,5-T","2,4,5-TP - PHENOXY ACID","2,4,5-TRICHLOROPHENOL","2,4,6-TRICHLOROPHENOL","2,4-D","2,4-DB","2,4-DICHLOROPHENOL","2,4-DIMETHYLPHENOL","2,4-DINITROPHENOL","2,4-DINITROTOLUENE","2,4-DP (DICHLORPROP)","2,6-DIETHYLANILE","2,6-DIMETHYLNAPHTHALENE","2,6-DINITROTOLUENE",2-CHLORONAPHTHALENE,2-CHLOROPHENOL,2-HEXANONE,"2-METHYL-4,6-DINITROPHENOL",2-METHYLFLUORANTHENE,2-METHYLNAPHTHALENE,2-METHYLPHENANTHRENE,2-NITROANILINE,2-NITROPHENOL,"3,3'-DICHLOROBENZIDINE",3-HYDROXYCARBOFURAN,3-METHYLPHENANTHRENE,3-NITROANILINE,4-BROMOPHENYL PHENYL ETHER,4-CHLORO-3-METHYLPHENOL,4-CHLOROANILINE,4-CHLOROPHENYL PHENYL ETHER,4-METHYL PHENOL,4-METHYLCHRYSENE,4-NITROANILINE,4-NITROPHENOL,4H-CYCLOPENTA[DEF]PHENANTHRENE,ACENAPHTHENE,ACENAPHTHYLENE,ACETOCHLOR,ACETONE,ACID VOLATILE SULFIDE,ALACHLOR,ALDICARB,ALDICARB SULFONE,ALDICARB SULFOXIDE,ALDRIN,ALUMINUM,AMETRYN,ANILINE,ANTHRACENE,ANTIMONY,AROCHLOR,"AROCLOR, PCB",ARSENIC,ARSENIC (III+V),ARSENITE (As III),ATRAZINE,AVS MINUS SEM,AZINPHOS METHYL,AZOBENZENE,Aroclor 5460,BARIUM,BDE (TOTAL),BDE-100,BDE-100 + PCB-194,BDE-153,BDE-154,BDE-47,BDE-99,BENEFIN,BENZENE,BENZIDINE,BENZO(B+K)FLUORANTHENE,BENZOIC ACID,BENZONAPHTHOTHIOPHENE,BENZO[A]FLUORENE,BENZO[A]PYRENE,BENZO[B]FLUORANTHENE,BENZO[B]FLUORENE,BENZO[C]PHENANTHRENE,BENZO[E]PYRENE,BENZO[GHI]FLUORANTHENE,BENZO[GHI]PERYLENE,BENZO[J]FLUORANTHENE,BENZO[K]FLUORANTHENE,BENZYL ALCOHOL,BENZ[A]ANTHRACENE,BERYLLIUM,BHC-ALPHA,BHC-BETA,BHC-DELTA,BHC-GAMMA (LINDANE),BIBENZYlL,BILE BENZO,BILE NAPH,BILE PHEN,BILE TPAH,BIPHENYL,"BIPHENYL, 4-METHYL",BIS (2-CHLOROISOPROPYL) ETHER,"BIS(2,4,6-TRICHLOROPHENYL) UREA (TCPU)",BIS(2-CHLOROETHOXY)METHANE,BIS(2-CHLOROETHYL)ETHER,BIS(TRI-N-BUTYLTIN)OXIDE,BORON,"BROMKAL,PENTABROMO-",BROMODICHLOROMETHANE,BROMOFORM,BUTYL BENZYL PHTHALATE,BUTYLATE,"Benzenehexachloride,alpha-","Benzenehexachloride,beta-","Benzenehexachloride,gamma-",C1 CHRYSENE,C1 DIBENZOTH,C1 FLUORENE,"C1-Dibenzo(a,h)anthracene",C1-Fluoranthenes & Pyrenes,C1-NAPHTHALENE,C1-Naphthobenzothiophene,C1-Phenanthrenes & Anthracenes,C1_Fluoranthene/Anthracene,C2 CHRYSENE,C2 DIBENZOTH,C2 FLUORENE,"C2-Dibenzo(a,h)anthracene",C2-Fluoranthenes/Pyrenes,C2-NAPHTHALENE,C2-Naphthobenzothiophene,C2-Phenanthrenes & Anthracenes,C2_Fluoranthene/Anthracene,C3 CHRYSENE,C3 DIBENZOTH,"C3-Dibenzo(a,h)anthracene",C3-FLUORENE,C3-Fluoranthenes/Pyrenes,C3-NAPHTHALENE,C3-Naphthobenzothiophene,C3-Phenanthrenes & Anthracenes,C4 CHRYSENE,C4-NAPHTHALENE,C4-Phenanthrenes & Anthracenes,CADMIUM,CALCIUM,CALCIUM CARBONATE,CARBARYL (SEVIN),CARBAZOLE,CARBOFURAN,CARBON,CARBON DISULFIDE,CARBON TETRACHLORIDE,CARBOPHENOTHION,CHLORDANE,CHLORDANE MIX,CHLORDANE-ALPHA,CHLORDANE-CIS,CHLORDANE-GAMMA,CHLORDANE-TRANS,CHLORIDE,CHLOROBENZENE,"CHLOROBIPHENYL, 2,3,3,4,4',5,5","CHLOROBIPHENYL, POLY-",CHLOROETHANE,CHLOROFORM,CHLOROMETHANE,CHLOROTHALONIL,CHLORPYRIFOS,CHLORPYRIFOS-METHYL,CHROMIUM,CHROMIUM (VI),"CHROMIUM III, TOTAL",CHRYSENE,CHRYSENE PLUS TRIPHENYLENE,CHRYSENE-BENZ[A]ANTHRACENE,CIS- AND TRANS-PERMETHRIN,"CIS-1,3-DICHLOROPROPENE",CIS-PERMETHRIN,"CLAY, P.S. = <.0039",COBALT,COPPER,CYANAZINE,Chlordane (1),Chlordane C,Chlordane cpd,"Chlordane cpd + DDE,2,4'- + PCB-84","Chlordane cpd + PCB-37,42,59",Chlordane cpd + PCB-44,Chlordane cpd + PCB-84,Chlordane(1),Chlordane(3),Chlordane(5),"Chlordane(5) + DDE,2,4'- + PCB-84",Chlordane(5) + PCB-84,Chlordane(7),"Chlordane, cis- + PCB-99","Chlordane, trans- + PCB-84","Chlordane, trans- + PCB-92","Chlordane,c

## Merge buoy and water quality data

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [25]:
buoypath = "../data/to_merge_buoy.csv"
data_buoy = pd.read_csv(buoypath)

waterpath = "../data/to_merge_water.csv"
data_water = pd.read_csv(waterpath)

In [26]:
data_buoy['Sample_datetime'] = pd.to_datetime(dict(year=data_buoy.Sample_year,month= data_buoy.Sample_month,day=data_buoy.Sample_day,hour=data_buoy.Sample_hour,minute=data_buoy.Sample_minute,second=data_buoy.Sample_second))

data_water['Sample_datetime'] = pd.to_datetime(dict(year=data_water.Sample_year,month= data_water.Sample_month,day=data_water.Sample_day,hour=data_water.Sample_hour,minute=data_water.Sample_minute,second=data_water.Sample_second))

# data_buoy.set_index('Sample_datetime',inplace=True)
# data_buoy_hourly = data_buoy.resample('H').mean()

# data_water.set_index('Sample_datetime',inplace=True)
# data_water_hourly = data_water.resample('H').mean()

In [27]:
data_merge = pd.merge(data_buoy,data_water, on = ['Sample_datetime','Latitude','Longitude'], how = 'outer')

In [28]:
data_merge = data_merge.drop(columns = ['index','Sample_year_x', 'Sample_month_x','Sample_day_x','Sample_hour_x','Sample_minute_x','Sample_second_x','Sample_year_y', 'Sample_month_y','Sample_day_y','Sample_hour_y','Sample_minute_y','Sample_second_y'])

# helper functions
def combine_date_time_strings(df):
    # Convert the combined string to datetime
    datetime_col = "Sample"

    df[datetime_col + "_year"] = df['Sample_datetime'].dt.year
    df[datetime_col + "_month"] = df['Sample_datetime'].dt.month
    df[datetime_col + "_day"] = df['Sample_datetime'].dt.day
    df[datetime_col + "_hour"] = df['Sample_datetime'].dt.hour
    df[datetime_col + "_minute"] = df['Sample_datetime'].dt.minute
    df[datetime_col + "_second"] = df['Sample_datetime'].dt.second
    return df

data_merge = combine_date_time_strings(data_merge)

In [32]:
data_merge.to_csv('../data/plank_Chesapeake_buoywater_merged.csv', index=False)

## Split merged data into seasons

In [33]:
# winter : Dec - Feb
# spring : Mar - May
# summer : Jun - Aug
# fall   : Sep - Nov

def get_season(month):
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    elif month in [9, 10, 11]:
        return 'fall'

data_merge['season'] = data_merge['Sample_month'].apply(get_season)


In [34]:
winter = data_merge[data_merge['season'] == 'winter']
spring = data_merge[data_merge['season'] == 'spring']
summer = data_merge[data_merge['season'] == 'summer']
fall = data_merge[data_merge['season'] == 'fall']

winter.to_csv('../data/plank_Chesapeake_buoywater_merged_winter.csv', index=False)
spring.to_csv('../data/plank_Chesapeake_buoywater_merged_spring.csv', index=False)
summer.to_csv('../data/plank_Chesapeake_buoywater_merged_summer.csv', index=False)
fall.to_csv('../data/plank_Chesapeake_buoywater_merged_fall.csv', index=False)

## XGBoost and SHAP for seasons

In [1]:
import shap
import xgboost
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
# import cupy as cp


ModuleNotFoundError: No module named 'xgboost'

In [4]:
!pip install xgboost

  Using cached xgboost-2.1.1-py3-none-manylinux_2_28_x86_64.whl.metadata (2.1 kB)
  Using cached nvidia_nccl_cu12-2.22.3-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━ 140.5/153.9 MB 3.6 MB/s eta 0:00:04

In [ ]:
!pip3 install xgboost